# Python Notebook Demonstrating Red Channel (a-SMA) Processing

Here, we detail an example of using our developed modules to process in batch our red channel images.
We begin by importing a number of packages that are required. Namely, we need to import the standard functions, such as NumPy, Pandas, Matplotlib. Additionally, we need to import modules from our cells_in_gel package and some skimage functions to ensure that our functions work.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from skimage import io
from skimage.measure import regionprops, regionprops_table
from skimage.morphology import disk

from cells_in_gel import preprocess as pp
from cells_in_gel import properties as props
from cells_in_gel import batch

We next create a list of our images using the preprocessing function `list_of_images` and create a dict of max projections using the batch function `max_projection` as we would like to keep our image titles as keys associated with their respective images.

In [ ]:
path = '$$$$PATHNAME HERE$$$$' #change with pathname.
files = pp.list_of_images('C4', path)

In [ ]:
max_proj = batch.max_projection(files)

Next, we utilize the preprocessing function `SMA_segment` to segment and create labels of our red channel, a functional similar to `phalloidin_segment` but with specific presets for the red channel. These labels can be then used as inputs to the properties function `im_properties` to create a regionprops table. 

In [ ]:
labels = {}
overlay = {}
regions = {}
   
for file in files:
    labels[file], overlay[file] = pp.SMA_segment(max_proj[file])

for file in files:   
    regions[file] = props.im_properties(labels[file], max_proj[file])

Let's print the regions table to make sure we've gotten what we want. Here, we can see that our code has segemented each "SMA" blob into its own blob and has taken values specified by the function. Here, we are mainly interested in mean_intensity, so we will disregard the other values as we progress.

In [ ]:
print(regions[files[0]].head())

We then proceed to calculate the average mean intensity and standard deviations for each max projection, again making new dicts to ensure that we still have the image titles as the keys.

In [ ]:
avg_props = {}
var_props = {}

for file in files:
    avg_props[file] = np.mean(regions[file]['mean_intensity'])
    var_props[file] = np.std(regions[file]['mean_intensity'])

We can see that our function worked, and that calling on the first image of the dictionary, we see that image has an average intensity value of 2418.62.

In [ ]:
print(avg_props[files[0]])

Next, we call upon our `mergeDict` function from the batch file to merge the two dictionaries to make a summary dict which is then converted to an array called summary. But we have an issue! The image titles are now column titles, which is not what we desire.

In [ ]:
summary = pd.DataFrame.from_dict(batch.mergeDict(var_props, avg_props))
summary.head()

So we transpose our array, and label the columns, and while we're at it also change the name of the index.

In [ ]:
summary = summary.transpose()
summary.columns = ['Mean Intensity', 'Standard Deviation']

In [ ]:
summary.index.name = 'type of cells'

In [ ]:
summary = summary.reset_index()
summary.head()

Now we have all of our information, yet our cell names are not great. We need to extract the information from the files such as Fibroblast and ECM genotype, Concentrations of ECM and RGD. To do this, we import the `re` package from python to split the strings located in the column "*type of cells*". This part is a bit complex, and highly depends on how you choose to label your files. Ideally if I were to do this, I would label all my tif files as such:

Channel#_FibroblastGenotype_ECMGenotype_ECMConcentration_RGDConcentration_Magnification_SampleNumber.

i.e. a file would be named as such: **C4_I61QTTA-CFbs_NTG_5_1mM_20x_002.tif** and we would break it at every '_'.

***However, our labmate had a bit of a weirder filing system, so hence there is a lot of splitting of strings***. His are labelled as such: **C4-I61QTTA-CFbs_NTG5ECM_1mMRGD_20x_002.tif**, which is fine for looking at files, but not so much for a consistent pattern of delimiters.

In [ ]:
import re

In [ ]:
copymod = summary.copy()
copymod['Fibroblast Genotype'] = "d"
copymod['ECM Genotype'] = "d"
copymod['ECM Concentration mg/mL'] = "d"
copymod['RGD Concentration mM'] ="d"

"""HIGHLY DEPENDENT ON HOW YOU LABEL YOUR FILES AND WHAT MICE YOU ARE USING.
    We need to break apart file name to label the columns properly so we can sort later.
    File name structure in this case: C4-FibroblastGenotype_ECMGenotypeAndConcentration_RGDConcentration
    deletes the rest of the path name."""

for i, row in enumerate(summary['type of cells']):
    mod = re.split('C4-|_20x', row)   #Deletes everything aside from FGenotype_ECMGenotypeConc_RGDConc
    newmod = re.split('_|mM', mod[1]) #splits string at _ and mM
    genotype = re.split('(\d+)ECM',newmod[1]) #further splits second string of newmod to get the genotype of ECM by splitting at case of # and ECM
    concentration = re.split('[QG]|ECM', newmod[1]) #Splits newmod at instance of Q or G AND before ECM to get the number in between
    
    copymod['type of cells'].iat[i] = mod[1] #assigning strings to columns.
    copymod['Fibroblast Genotype'].iat[i] = newmod[0]
    copymod['ECM Genotype'].iat[i] = genotype[0]
    copymod['ECM Concentration mg/mL'].iat[i] = concentration[1]
    copymod['RGD Concentration mM'].iat[i] = newmod[2]
    
copymod['ECM Concentration mg/mL'].iat[4] = 0  #This is unfortunately an exception in how the files were labelled.

Let's check out our final table:

In [ ]:
copymod

Now it's time to graph our results using the groupby functionality! Since this data set has a lot of types of samples, but very little repetition of experiments. Here, I grouped by all the possible combinations.

In [ ]:
pretty = copymod.groupby(['Fibroblast Genotype','ECM Genotype','ECM Concentration mg/mL' ,'RGD Concentration mM' ]).mean().unstack(fill_value=0)
fibroblasts = pretty.groupby(['Fibroblast Genotype'])

errors = copymod.groupby(['Fibroblast Genotype', 'ECM Genotype', 'ECM Concentration mg/mL', 'RGD Concentration mM']).std().unstack(fill_value =0)

fibroblasts.plot.bar(y = ['Mean Intensity'], yerr = errors, capsize = 3)
plt.xlabel('Cell Type', size =15)
plt.ylabel('Mean Intensity', size = 15)
plt.legend(["0mM RGD","1mM RGD"])
plt.show()

But say I just want to look at how the different fibroblast populations responded to the RGD concentrations and disregard the rest, I could group my data this way as well.

In [ ]:
pretty2 = copymod.groupby(['Fibroblast Genotype','RGD Concentration mM' ]).mean().unstack(fill_value=0)
errors2 = copymod.groupby(['Fibroblast Genotype', 'RGD Concentration mM']).std().unstack(fill_value =0)
ax = pretty2.plot.bar(y = ['Mean Intensity'], yerr = errors2, capsize = 3)
plt.xlabel('Cell Type', size =15)
plt.ylabel('Mean Intensity', size = 15)
ax.legend(["O mM RGD", "1 mM RGD"])
plt.show()